<a href="https://colab.research.google.com/github/ehddnr301/dacon_motion/blob/master/motion_pytorch_hrnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!rm -rf /content/gdrive/MyDrive/dacon-motion/train_imgs_folder

In [3]:
!rm -rf /content/gdrive/MyDrive/dacon-motion/test_imgs_folder

In [4]:
!pip install torchinfo

In [ ]:
!pip install albumentations==0.4.6

     |██████████████████████████████▊ | 112kB 803kB/s eta 0:00:01

In [ ]:
!pip install -U albumentations_experimental

In [ ]:
!git clone https://github.com/ehddnr301/pytorch-image-models.git

In [ ]:
!mv pytorch-image-models timmm

In [ ]:
!cd timmm && pip install .

In [ ]:
from timmm import timm

In [ ]:
!unzip -n '/content/gdrive/MyDrive/dacon-motion/train_imgs.zip' -d '/content/gdrive/MyDrive/dacon-motion/train_imgs_folder'

In [ ]:
!unzip -n '/content/gdrive/MyDrive/dacon-motion/test_imgs.zip' -d '/content/gdrive/MyDrive/dacon-motion/test_imgs_folder'

In [ ]:
import os
from typing import Tuple, Sequence, Callable
import csv
import cv2
import random
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
import torch
import torch.optim as optim
from torch import nn, Tensor
from torch.utils.data import Dataset, DataLoader, random_split
from torchinfo import summary
import matplotlib.pyplot as plt


import albumentations
from albumentations.pytorch import ToTensorV2, ToTensor
from albumentations_experimental import HorizontalFlipSymmetricKeypoints

from torchvision import transforms
import timm

In [ ]:
# random seed
random_seed = 301
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)


IMAGE_HEIGHT = 1024
IMAGE_WIDTH = 1024
IMAGE_CHANNELS=3
EPOCHS=30
BATCH_SIZE = 4

PATH_TRAIN_DATASET='/content/gdrive/MyDrive/dacon-motion/train_imgs_folder/'
PATH_TEST_DATASET='/content/gdrive/MyDrive/dacon-motion/test_imgs_folder/'
PATH_TRAIN_ANS_CSV='/content/gdrive/MyDrive/dacon-motion/train_df.csv'
SUB_DF = '/content/gdrive/MyDrive/dacon-motion/sample_submission.csv'
PATH_TRAIN_CSV='/content/gdrive/MyDrive/dacon-motion/split_train_df.csv'
PATH_VALID_CSV='/content/gdrive/MyDrive/dacon-motion/split_val_df.csv'

In [ ]:
df = pd.read_csv(PATH_TRAIN_ANS_CSV)
df.head(2)

In [ ]:
error_list=[317, 869, 873, 877, 911, 1559, 1560, 1562, 1566, 1575, 1577, 1578, 1582, 1606, 1607, 1622, 1623, 1624, 1625, 1629, 3968, 4115, 4116, 4117, 4118, 4119, 4120, 4121, 4122, 4123, 4124, 4125, 4126, 4127, 4128, 4129, 4130, 4131, 4132, 4133, 4134, 4135, 4136, 4137, 4138, 4139, 4140, 4141, 4142, 4143, 4144, 4145, 4146, 4147, 4148, 4149, 4150, 4151, 4152, 4153, 4154, 4155, 4156, 4157, 4158, 4159, 4160, 4161, 4162, 4163, 4164, 4165, 4166, 4167, 4168, 4169, 4170, 4171, 4172, 4173, 4174, 4175, 4176, 4177, 4178, 4179, 4180, 4181, 4182, 4183, 4184, 4185, 4186, 4187, 4188, 4189, 4190, 4191, 4192, 4193, 4194]

In [ ]:
df.drop(error_list, inplace=True)

In [ ]:
df

In [ ]:
x_coor = [col for col in df.columns if '_x' in col]
y_coor = [col for col in df.columns if '_y' in col]

In [ ]:
val_name_list = []
check_list = []

In [ ]:
def spliting(x):
  x1 = x.split('000')[0]
  if check_list.count(x1) < 1:
    check_list.append(x1)
    val_name_list.append(x)

In [ ]:
df['image'].apply(spliting)

In [ ]:
val_name_list

In [ ]:
val_df = df[df['image'].isin(val_name_list)]

In [ ]:
train_df = df[~df['image'].isin(val_name_list)]

In [ ]:
val_df.head(2)

In [ ]:
train_df.head(2)

In [ ]:
train_imgs = train_df.iloc[:, 0].to_numpy()
motions = train_df.iloc[:, 1:]
columns = motions.columns.to_list()[::2]
class_labels = [label.replace('_x', '').replace('_y', '') for label in columns]
train_keypoints = []
for motion in motions.to_numpy():
    a_train_keypoints = []
    for i in range(0, motion.shape[0], 2):
        a_train_keypoints.append((float(motion[i]), float(motion[i+1])))
    train_keypoints.append(a_train_keypoints)
train_keypoints = np.array(train_keypoints)

In [ ]:
val_imgs = val_df.iloc[:, 0].to_numpy()
motions = val_df.iloc[:, 1:]
columns = motions.columns.to_list()[::2]
class_labels = [label.replace('_x', '').replace('_y', '') for label in columns]
val_keypoints = []
for motion in motions.to_numpy():
    a_val_keypoints = []
    for i in range(0, motion.shape[0], 2):
        a_val_keypoints.append((float(motion[i]), float(motion[i+1])))
    val_keypoints.append(a_val_keypoints)
val_keypoints = np.array(val_keypoints)

In [ ]:
len(val_keypoints), len(val_imgs)

In [ ]:
len(train_keypoints), len(train_imgs)

In [ ]:
class_labels

In [ ]:
from sklearn.model_selection import train_test_split

# train, valid = train_test_split(df, test_size=0.05, random_state=random_seed)

In [ ]:
class MotionDataset(Dataset):
    """__init__ and __len__ functions are the same as in TorchvisionDataset"""
    def __init__(self, data_dir, imgs, keypoints, class_labels=None, data_transforms=None):
        self.data_dir = data_dir
        self.imgs = imgs
        self.keypoints = keypoints
        self.class_labels = class_labels
        self.data_transforms = data_transforms

    def __getitem__(self, idx):
        # Read an image with OpenCV
        img = cv2.imread(os.path.join(self.data_dir, self.imgs[idx]))
        # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        keypoints = self.keypoints[idx]
    
        if self.data_transforms:
            augmented = self.data_transforms(image=img, keypoints=keypoints, class_labels=self.class_labels)
            img = augmented['image']
            keypoints = augmented['keypoints']
            keypoints = [(x/1024, y/1024) for x,y in keypoints]
            # keypoints = [(x, y) for x,y in keypoints]

        keypoints = np.array(keypoints).flatten()

        return img, keypoints
    
    def __len__(self):
        return len(self.imgs)

In [ ]:
class TestDataset(Dataset):
    """__init__ and __len__ functions are the same as in TorchvisionDataset"""
    def __init__(self, data_dir, imgs, data_transforms=None):
        self.data_dir = data_dir
        self.imgs = imgs
        self.data_transforms = data_transforms

    def __getitem__(self, idx):
        filename = self.imgs[idx]
        # Read an image with OpenCV
        img = cv2.imread(os.path.join(self.data_dir, self.imgs[idx]))
        # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        if self.data_transforms:
            augmented = self.data_transforms(image=img)
            img = augmented['image']

        return filename, img
    
    def __len__(self):
        return len(self.imgs)
    


In [ ]:
transforms_train = albumentations.Compose([
      albumentations.Crop(426, 56, 1450, 1080),
      albumentations.OneOf([
                HorizontalFlipSymmetricKeypoints(symmetric_keypoints={
                    (0,0),(1,2),(3,4),(5,6),(7,8),(9,10),(11,12),(13,14),(15,16),(17,17),(18,19),(20,20), (21,21),(22,23)
                }, p=1),
                albumentations.Rotate(limit=45, p=1),
        ], p=0.5),
      albumentations.OneOf([                    
          albumentations.RandomBrightness(p=1),
          albumentations.MotionBlur(blur_limit=[3,20], p=0.5),
      ], p=0.5),
      albumentations.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
      ToTensorV2()
], keypoint_params=albumentations.KeypointParams(format='xy', label_fields=['class_labels'], remove_invisible=False))

transforms_valid = albumentations.Compose([
    albumentations.Crop(426, 56, 1450, 1080),
    albumentations.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ToTensorV2()
], keypoint_params=albumentations.KeypointParams(format='xy', label_fields=['class_labels'], remove_invisible=False))

transforms_test = albumentations.Compose([
    albumentations.Crop(426, 56, 1450, 1080),
    albumentations.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ToTensorV2()
])


In [ ]:
# trainset = MotionDataset(PATH_TRAIN_DATASET, PATH_TRAIN_CSV, transforms_train)
# valset = MotionDataset(PATH_TRAIN_DATASET, PATH_VALID_CSV, transforms_valid)
# testset = MotionDataset(PATH_TEST_DATASET, SUB_DF, transforms_test)
test_imgs = os.listdir(PATH_TEST_DATASET)
trainset = MotionDataset(PATH_TRAIN_DATASET, train_imgs, train_keypoints, data_transforms=transforms_train, class_labels=class_labels)
valset = MotionDataset(PATH_TRAIN_DATASET, val_imgs, val_keypoints, data_transforms=transforms_valid, class_labels=class_labels)
testset = TestDataset(PATH_TEST_DATASET, test_imgs, data_transforms=transforms_test)

train_loader = DataLoader(trainset, batch_size=4)
val_loader = DataLoader(valset, batch_size=1)
test_loader = DataLoader(testset, batch_size=1, shuffle=False)

In [ ]:
# sample, keypointss = trainset[0]
sample, _ = trainset[205]

In [ ]:

# keypointss = keypointss * 1024

In [ ]:
def custom_imshow(img):
  img = img.numpy()
  plt.figure(figsize=(12,12))
  plt.imshow(np.transpose(img, (1, 2, 0)))
  plt.show()
# def custom_imshow(img, keypointss):
#   print(keypointss)
#   img = img.numpy()
#   plt.figure(figsize=(12,12))
#   for j in range(0,len(keypointss),2):
#     plt.plot(keypointss[j], keypointss[j+1],'rx')
#   plt.imshow(np.transpose(img, (1, 2, 0)))
    

In [ ]:
# custom_imshow(sample,keypointss)
custom_imshow(sample)

In [ ]:
class HRnetModel(nn.Module):
    def __init__(self, num_classes=48, model_name='hrnet_w32', pretrained=True):
        super(HRnetModel, self).__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained)
        self.model.classifier = nn.Linear(2048, num_classes)
        
    def forward(self, x):
        x = self.model(x)

        return x

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = HRnetModel().to(device)

# 16번째 이후에 끊겨서 8epoch부터 다시 불러서 학습
# PATH = '/content/gdrive/MyDrive/dacon-motion/checkpoint/model_hrnet_0321_16.pt'
# model = HRnetModel().to(device)

# checkpoint = torch.load(PATH)
# model.load_state_dict(checkpoint['model_state_dict'])


In [ ]:
print(summary(model, input_size=(1, 3, 1024, 1024), verbose=0))

In [ ]:
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0.001)
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
# criterion = nn.MSELoss(reduction='sum')
criterion = nn.MSELoss()
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer,
                                                  mode='min',
                                                  factor=0.5,
                                                  patience=2,)



for epoch in range(EPOCHS):
    

    for i, (images, targets) in enumerate(train_loader):

        optimizer.zero_grad()

        # images = images.float().to(device)
        images = images.to(device)
        targets = targets.to(device)
        model.train()
        with torch.set_grad_enabled(True):
            outputs = model(images)
            loss = criterion(outputs.float(), targets.float())
            loss.backward()
            optimizer.step()

        if i % 100 == 0:
            outputs = outputs
            
            print(f'{epoch+1}: {loss.item():.5f}')


    with tqdm(val_loader,
            total=val_loader.__len__(),
            unit="batch") as valid_bar:
        for i, (images, targets) in enumerate(valid_bar):
            valid_bar.set_description(f"Valid Epoch {epoch + 1}")
            optimizer.zero_grad()

            images = images.to(device)
            targets = targets.to(device)

            # 모델의 dropoupt, batchnormalization를 eval모드로 설정
            # .forward()에서 중간 노드의 gradient를 계산

            model.eval()

            with torch.no_grad():
                # validation loss만을 계산
                probs  = model(images)
                valid_loss = criterion(probs.float(), targets.float())

            valid_bar.set_postfix(valid_loss = valid_loss.item())

    scheduler.step(valid_loss)


    if (epoch + 1) % 2 == 0:
        torch.save({
                'epoch': epoch+1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': loss,
                }, f'/content/gdrive/MyDrive/dacon-motion/checkpoint/model_hrnet_0326_{epoch + 1}.pt')
    
    print('------------------')

In [ ]:
# # 불러와서 예측하기

# PATH = '/content/gdrive/MyDrive/dacon-motion/checkpoint/model_hrnet_0325_16.pt'
# model = HRnetModel().to(device)

# checkpoint = torch.load(PATH)
# model.load_state_dict(checkpoint['model_state_dict'])


In [ ]:
!nvidia-smi

In [ ]:
all_predictions = []
files = []
with torch.no_grad():
    for filenames, inputs in tqdm(test_loader):
        predictions = list(model(inputs.to(device)).cpu().numpy())
        files.extend(filenames)
        for prediction in predictions:
            all_predictions.append(prediction)

In [ ]:
all_predictions[0]

In [ ]:
all_predictions[0].shape

In [ ]:
files[0]

In [ ]:
all_predictions = np.array(all_predictions)
for i in range(all_predictions.shape[0]):
    # all_predictions[i, [2*j for j in range(24)]] *= 1024
    # all_predictions[i, [2*j + 1 for j in range(24)]] *= 1024
    all_predictions[i, [j for j in range(48)]] *= 1024

In [ ]:
# submit * 1024 해주어야함.

In [ ]:
all_predictions[0]

In [ ]:
submit = pd.read_csv(SUB_DF)
submit.head(2)

In [ ]:
submit = pd.DataFrame(columns=submit.columns)
submit['image'] = files
submit.iloc[:, 1:] = all_predictions
submit.head()

In [ ]:
# inversed = scaler.inverse_transform(submit.iloc[:,1:])
# print(inversed[0])

In [ ]:
# submit.iloc[:,1:] = inversed
# submit.head(2)


In [ ]:
submit[x_coor] = (submit[x_coor] + 426)
submit[y_coor] = (submit[y_coor] + 56)
# submit[x_coor] = (submit[x_coor] * 2 + 426)
# submit[y_coor] = (submit[y_coor] * 2 + 56)

In [ ]:
submit.head(2)

In [ ]:
submit.to_csv('/content/gdrive/MyDrive/dacon-motion/submit_hr_0326.csv', index=False)

In [ ]:
import glob


In [ ]:
plt.figure(figsize=(40,20))
count=1
test_paths = glob.glob(os.path.join(PATH_TEST_DATASET,'*.jpg'))

for i in np.random.randint(0,len(test_paths),5):
    
    plt.subplot(5,1, count)
    
    img_sample_path = test_paths[i]
    img_name = img_sample_path.split('/')[6]
    img = Image.open(img_sample_path)
    img_np = np.array(img)
    key = submit[submit['image'] == img_name].iloc[0, 1:49]

    keypoint = submit.iloc[:,1:49] #위치키포인트 하나씩 확인

    for j in range(0,len(keypoint.columns),2):
        plt.plot(key[j], key[j+1],'rx')
        plt.imshow(img_np)
    
    count += 1